# *tridesclous* example with olfactory bulb dataset



In [1]:
%matplotlib inline

import time
import numpy as np
import matplotlib.pyplot as plt
import tridesclous as tdc

from tridesclous import DataIO, CatalogueConstructor, Peeler

# DataIO = define datasource and working dir

trideclous provide some datasets than can be downloaded.

Note this dataset contains 2 trials in 2 different files. (the original contains more!)

Each file is considers as a *segment*. *tridesclous* automatically deal with it.

Theses 2 files are in **RawData** format this means binary format with interleaved channels.


In [2]:
#download dataset
localdir, filenames, params = tdc.download_dataset(name='olfactory_bulb')
print(filenames)
print(params)

print()
#create a DataIO
import os, shutil
dirname = 'tridesclous_olfactory_bulb'
if os.path.exists(dirname):
    #remove is already exists
    shutil.rmtree(dirname)    
dataio = DataIO(dirname=dirname)

# feed DataIO
dataio.set_data_source(type='RawData', filenames=filenames, **params)

dataio.add_one_channel_group(channels=range(14))

print(dataio)

    
    

['/home/samuel/Documents/projet/tridesclous/example/olfactory_bulb/OB_file1.raw', '/home/samuel/Documents/projet/tridesclous/example/olfactory_bulb/OB_file2.raw', '/home/samuel/Documents/projet/tridesclous/example/olfactory_bulb/OB_file3.raw']
{'dtype': 'int16', 'sample_rate': 10000.0, 'total_channel': 16, 'bit_to_microVolt': 0.30517578125}

DataIO <id: 139704555837144> 
  workdir: tridesclous_olfactory_bulb
  sample_rate: 10000.0
  total_channel: 16
  channel_groups: 0 [ch0 ch1 ch2 ch3 ... ch10 ch11 ch12 ch13]
  nb_segment: 3
  length: 150000 150000 150000
  durations: 15.0 15.0 15.0 s.


# CatalogueConstructor

In [7]:
cc = CatalogueConstructor(dataio=dataio)
print(cc)

CatalogueConstructor
  chan_grp 0 - ch0 ch1 ch2 ... ch12 ch13
  Signal pre-processing not done yet


## Use automatic parameters and apply the whole chain

tridesclous propose an automatic parameters choice and can apply in one function all the steps.


In [8]:
from pprint import pprint
params = tdc.get_auto_params_for_catalogue(dataio, chan_grp=0)
pprint(params)

{'adjacency_radius_um': 400.0,
 'chunksize': 1000,
 'clean_cluster': False,
 'clean_cluster_kargs': {},
 'clean_waveforms': {'alien_value_threshold': None},
 'cluster_kargs': {'max_loop': 1000, 'min_cluster_size': 20},
 'cluster_method': 'pruningshears',
 'duration': 300.0,
 'extract_waveforms': {'mode': 'rand',
                       'nb_max': 20000,
                       'wf_left_ms': -1.5,
                       'wf_right_ms': 2.5},
 'feature_kargs': {'n_components_by_channel': 3},
 'feature_method': 'pca_by_channel',
 'mode': 'sparse',
 'noise_snippet': {'nb_snippet': 300},
 'peak_detector': {'engine': 'opencl',
                   'method': 'geometrical',
                   'peak_sign': '-',
                   'peak_span_ms': 0.7,
                   'relative_threshold': 5.0},
 'preprocessor': {'common_ref_removal': False,
                  'engine': 'numpy',
                  'highpass_freq': 300.0,
                  'lostfront_chunksize': -1,
                  'lowpass_freq': 50

## apply all catalogue steps

In [9]:
tdc.apply_all_catalogue_steps(cc, params, verbose=True)

estimate_signals_noise 0.1705890859993815
run_signalprocessor 0.8053319609998653
extract_some_waveforms 0.7399420740002824
clean_waveforms 0.02303425399986736
extract_some_noise 0.0693022049999854
project 0.6348587969996515
find_clusters 1.964479604999724


In [10]:
print(cc)

CatalogueConstructor
  chan_grp 0 - ch0 ch1 ch2 ... ch12 ch13
  nb_peak_by_segment: 4291, 3898, 4534
  some_waveforms.shape: (12720, 40, 14) (sparse 0.21)
  some_features.shape: (12720, 42)
  cluster_labels 13 [-11 -1 0 ... 11 12]



## Open CatalogueWindow for visual check

At the end we can save the catalogue.

In [12]:
%gui qt5
import pyqtgraph as pg
app = pg.mkQApp()
win = tdc.CatalogueWindow(catalogueconstructor)
win.show()
app.exec_()

CatalogueController 0
construct_catalogue 0.598916327000552


0

In [11]:
cc.make_catalogue_for_peeler()

make_catalogue 0.07650131300033536


# Peeler

Use automatic parameters.


In [14]:
peeler_params = tdc.get_auto_params_for_peelers(dataio, chan_grp=0)
pprint(peeler_params)

{'argmin_method': 'opencl',
 'engine': 'geometrical',
 'inter_sample_oversampling': True,
 'sparse_threshold_mad': 1.5,
 'use_sparse_template': True}


In [15]:
catalogue = dataio.load_catalogue()

peeler = Peeler(dataio)
peeler.change_params(catalogue=catalogue, **peeler_params)

t1 = time.perf_counter()
peeler.run()
t2 = time.perf_counter()
print('peeler.run', t2-t1)

print()
for seg_num in range(dataio.nb_segment):
    spikes = dataio.get_spikes(seg_num)
    print('seg_num', seg_num, 'nb_spikes', spikes.size)
    print(spikes[:3])
    


100%|██████████| 146/146 [00:07<00:00, 17.55it/s]

peeler.run 18.26728854799967

seg_num 0 nb_spikes 5093
[( 7,   5, -0.38943046) (14, -10,  0.        ) (28,   1, -0.36095695)]
seg_num 1 nb_spikes 4596
[( 62, -10,  0.       ) (147,   2, -0.2981372) (147, -10,  0.       )]
seg_num 2 nb_spikes 5650
[(45, -10,  0.        ) (45,   5, -0.30148764) (50,  11, -0.46073323)]


## Open PeelerWindow for visual checking

In [15]:
%gui qt5
import pyqtgraph as pg
app = pg.mkQApp()
win = tdc.PeelerWindow(dataio=dataio, catalogue=initial_catalogue)
win.show()
app.exec_()


0

Here a snappshot of PeelerWindow

<img src="../doc/img/snapshot_peelerwindow.png">